In [7]:
import requests
import pandas as pd
from datetime import datetime

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

In [2]:
base_url = "https://api.notion.com/v1/"
headers = {
    "Authorization": "secret_1RCbNJ5TqX2cWrLHTwVrKQr9NvdbyZfHGIMu8IPxuTm",
    "Content-Type": "application/json",
    "Notion-Version": "2022-06-28"
}

def retrieve_database(database_id):
    has_more = True
    next_cursor = None
    all_results = []
    url = f"{base_url}databases/{database_id}/query"

    while has_more:
        json_data = {"start_cursor": next_cursor} if next_cursor else {}
        response = requests.post(url, headers=headers, json=json_data)
        if response.status_code != 200:
            raise Exception(f"Erro ao buscar dados: {response.status_code}\n{response.text}")
        data = response.json()
        all_results.extend(data["results"])
        has_more = data["has_more"]
        next_cursor = data.get("next_cursor")

    return all_results

def monitoring_production(database_id, instructors, start="2024-01-01", end="2024-03-31"):
    database = retrieve_database(database_id)

    pages = []
    start = datetime.strptime(start, "%Y-%m-%d")
    end = datetime.strptime(end, "%Y-%m-%d")
    for page in database:
        start_page = datetime.strptime(page["properties"]["Período Realizado"]["date"]["start"], "%Y-%m-%d")
        end_page = datetime.strptime(page["properties"]["Período Realizado"]["date"]["end"], "%Y-%m-%d")
        if (start <= start_page <= end) or (start <= end_page <= end):
            instructor = page['properties']['Instrutor(a)']['rich_text'][0]['text']['content']
            if instructor in instructors:
                pages.append(page)

    return pages

pessoas = ["Afonso", "Allan", "Ana Duarte", "Bia", "Danielle", "Igor", "João", "Marcelo", "Mirla", "Val"]
database = monitoring_production("f7dcab16e74b4f61a7a9265508bab58d", pessoas)

In [3]:
df = pd.DataFrame(database)
df = pd.json_normalize(df['properties'])
df.head(3)

,Atividade.id,Atividade.type,Atividade.rich_text,Qualidade das entregas.id,Qualidade das entregas.type,Qualidade das entregas.select,Tempo Gasto (em Dias).id,Tempo Gasto (em Dias).type,Tempo Gasto (em Dias).number,Anotações.id,Anotações.type,Anotações.rich_text,Última Atualização.id,Última Atualização.type,Última Atualização.date.start,Última Atualização.date.end,Última Atualização.date.time_zone,Planejamento.id,Planejamento.type,Planejamento.select.id,Planejamento.select.name,Planejamento.select.color,Velocidade.id,Velocidade.type,Velocidade.select,Produto.id,Produto.type,Produto.rich_text,Id.id,Id.type,Id.rich_text,Instrutor(a).id,Instrutor(a).type,Instrutor(a).rich_text,Resposta aos feedbacks.id,Resposta aos feedbacks.type,Resposta aos feedbacks.select,Cumprimento de acordos.id,Cumprimento de acordos.type,Cumprimento de acordos.select,Comunicação.id,Comunicação.type,Comunicação.select,Assiduidade.id,Assiduidade.type,Assiduidade.select,Status.id,Status.type,Status.select.id,Status.select.name,Status.select.color,Dedicação.id,Dedicação.type,Dedicação.select,Período Realizado.id,Período Realizado.type,Período Realizado.date.start,Período Realizado.date.end,Período Realizado.date.time_zone,Título do Produto.id,Título do Produto.type,Título do Produto.rich_text,Colaboração.id,Colaboração.type,Colaboração.select,Flexibilidade.id,Flexibilidade.type,Flexibilidade.select,Período Planejado.id,Período Planejado.type,Período Planejado.date.start,Período Planejado.date.end,Período Planejado.date.time_zone,Código do Produto.id,Código do Produto.type,Código do Produto.rich_text,Tarefa.id,Tarefa.type,Tarefa.title,Qualidade das entregas.select.id,Qualidade das entregas.select.name,Qualidade das entregas.select.color,Velocidade.select.id,Velocidade.select.name,Velocidade.select.color,Resposta aos feedbacks.select.id,Resposta aos feedbacks.select.name,Resposta aos feedbacks.select.color,Cumprimento de acordos.select.id,Cumprimento de acordos.select.name,Cumprimento de acordos.select.color,Comunicação.select.id,Comunicação.select.name,Comunicação.select.color,Assiduidade.select.id,Assiduidade.select.name,Assiduidade.select.color,Dedicação.select.id,Dedicação.select.name,Dedicação.select.color,Colaboração.select.id,Colaboração.select.name,Colaboração.select.color,Flexibilidade.select.id,Flexibilidade.select.name,Flexibilidade.select.color
0,%3AXrD,rich_text,"[{'type': 'text', 'text': {'content': 'Suporte...",GJzq,select,NaN,Kua%5C,number,4,McZx,rich_text,"[{'type': 'text', 'text': {'content': '', 'lin...",NCp_,date,2024-03-25T17:03:00.000+00:00,None,None,Rsxg,select,1a482c98-3fd1-4292-b1f9-b750f7345307,Não Planejada,red,Wvzw,select,NaN,XwBT,rich_text,"[{'type': 'text', 'text': {'content': 'Demanda...",Y~%7DC,rich_text,"[{'type': 'text', 'text': {'content': '0741a6c...",Z_q%7C,rich_text,"[{'type': 'text', 'text': {'content': 'Marcelo...",cQ%3AT,select,NaN,jAlI,select,NaN,k~o%7B,select,NaN,qVjL,select,NaN,q%7DAN,select,6bdb39d5-a81d-4973-a01d-dccdbd567467,Finalizada,green,rSUO,select,NaN,ujf%7B,date,2024-03-25,2024-03-28,None,vqx%3E,rich_text,"[{'type': 'text', 'text': {'content': 'Imersão...",xmml,select,NaN,yfs%5D,select,NaN,%7Cbp%3B,date,2000-01-01,2000-01-01,None,%7Ct%40%3F,rich_text,"[{'type': 'text', 'text': {'content': '', 'lin...",title,title,"[{'type': 'text', 'text': {'content': 'Demanda...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,%3AXrD,rich_text,"[{'type': 'text', 'text': {'content': 'Suporte...",GJzq,select,NaN,Kua%5C,number,4,McZx,rich_text,"[{'type': 'text', 'text': {'content': '', 'lin...",NCp_,date,2024-03-25T17:03:00.000+00:00,None,None,Rsxg,select,1a482c98-3fd1-4292-b1f9-b750f7345307,Não Planejada,red,Wvzw,select,NaN,XwBT,rich_text,"[{'type': 'text', 'text': {'content': 'Demanda...",Y~%7DC,rich_text,"[{'type': 'text', 'text': {'content': '5jr65c7...",Z_q%7C,rich_text,"[{'type': 'text', 'text': {'content': 'Mirla',...",cQ%3AT,select,NaN,jAlI,select,NaN,k~o%7B,

In [ ]:
# df = pd.DataFrame(database)
# df = pd.json_normalize(df['properties'])
# for item in df.columns:
#     print(item)

In [13]:
df = pd.DataFrame(database)
df = pd.json_normalize(df['properties'])
df["Id"] = pd.json_normalize(df["Id.rich_text"].explode())["text.content"]
df["Instrutor(a)"] = pd.json_normalize(df["Instrutor(a).rich_text"].explode())["text.content"]
df["Status"] = df["Status.select.name"]
df["Tarefa"] = pd.json_normalize(df["Tarefa.title"].explode())["text.content"]
df["Período Planejado - start"] = df["Período Planejado.date.start"]
df["Período Planejado - end"] = df["Período Planejado.date.end"]
df["Período Realizado - start"] = df["Período Realizado.date.start"]
df["Período Realizado - end"] = df["Período Realizado.date.end"]
df["Atividade"] = pd.json_normalize(df["Atividade.rich_text"].explode())["text.content"]
df["Produto"] = pd.json_normalize(df["Produto.rich_text"].explode())["text.content"]
df["Planejamento"] = df["Planejamento.select.name"]
df["Código do Produto"] = pd.json_normalize(df["Código do Produto.rich_text"].explode())["text.content"]
df["Título do Produto"] = pd.json_normalize(df["Título do Produto.rich_text"].explode())["text.content"]

df["Qualidade das entregas"] = df["Qualidade das entregas.select.name"]
df["Velocidade"] = df["Velocidade.select.name"]
df["Resposta aos feedbacks"] = df["Resposta aos feedbacks.select.name"]
df["Cumprimento de acordos"] = df["Cumprimento de acordos.select.name"]
df["Comunicação"] = df["Comunicação.select.name"]
df["Assiduidade"] = df["Assiduidade.select.name"]
df["Dedicação"] = df["Dedicação.select.name"]
df["Colaboração"] = df["Colaboração.select.name"]
df["Flexibilidade"] = df["Flexibilidade.select.name"]

columns = ["Id", "Instrutor(a)", "Status", "Tarefa", "Período Planejado - start", "Período Planejado - end", "Período Realizado - start", "Período Realizado - end", "Atividade", "Produto", "Planejamento", "Código do Produto", "Título do Produto", "Qualidade das entregas", "Velocidade", "Resposta aos feedbacks", "Cumprimento de acordos", "Comunicação", "Assiduidade", "Dedicação", "Colaboração", "Flexibilidade"]
df = df[columns]
df = df.query('Status != "Cancelada"')
df = df.query('Produto != "FÉRIAS"')
df = df.query('Produto != "ATESTADO"')
df.shape
# df.head(3)

(139, 22)

In [14]:
table_aux = df.groupby(by=['Instrutor(a)', 'Título do Produto', 'Planejamento'])['Id'].count().reset_index()
table_aux

,Instrutor(a),Título do Produto,Planejamento,Id
0,Afonso,Como desenvolver soft skills para carreira de ...,Planejada,2
1,Afonso,Data Science,Não Planejada,1
2,Afonso,Excel (Marketing),Não Planejada,1
3,Afonso,IA aplicada a Data Science: utilizando IA como...,Planejada,5
4,Afonso,Tipos de visualizações,Planejada,1
5,Afonso,Utilizando IAs para análise exploratória de da...,Planejada,2
6,Afonso,Áreas de utilização do Google Big Query - Empr...,Planejada,1
7,Allan,Como as empresas estão utilizando IA na área d...,Planejada,2
8,Allan,Cultura Data Driven,Não Planejada,1
9,Allan,Data Driven,Não Planejada,1


In [15]:
# Quantidade de atividades planejadas e não planejadas por instrutor
table_01 = pd.pivot_table(table_aux, values='Título do Produto', index=['Instrutor(a)'], columns=['Planejamento'], aggfunc="count")
table_01.to_csv()

Planejamento,Não Planejada,Planejada
Instrutor(a),,
Afonso,2.0,5.0
Allan,4.0,5.0
Ana Duarte,2.0,5.0
Bia,1.0,6.0
Danielle,2.0,7.0
Igor,NaN,5.0
João,1.0,5.0
Marcelo,2.0,6.0
Mirla,2.0,1.0


In [16]:
table_aux = df.groupby(by=['Instrutor(a)', 'Produto', 'Título do Produto', 'Planejamento'])['Id'].count().reset_index()
table_aux = table_aux.groupby(by=['Instrutor(a)', 'Produto', 'Planejamento'])['Id'].count().reset_index()

In [17]:
# Quantidade de atividades planejadas e não planejadas por instrutor
table_02 = pd.pivot_table(table_aux, values='Produto', index=['Instrutor(a)', 'Produto'], columns=['Planejamento'], aggfunc="sum")
table_02

Planejamento                  Não Planejada  Planejada
Instrutor(a) Produto                                  
Afonso       Artigo                     1.0        2.0
             Curso                      NaN        1.0
             Formação                   1.0        NaN
             Podcast                    NaN        2.0
Allan        Artigo                     1.0        2.0
             Curso (E)                  NaN        2.0
             Formação                   1.0        NaN
             Palestra                   2.0        NaN
             Podcast                    NaN        1.0
Ana Duarte   Curso                      NaN        2.0
             Demanda Interna            1.0        NaN
             Podcast                    1.0        3.0
Bia          Alura+                     NaN        1.0
             Artigo                     NaN        2.0
             Curso                      NaN        1.0
             Imersão                    1.0        NaN
             Podcast                    NaN        2.0
Danielle     Artigo                     1.0        3.0
             Curso                      NaN        1.0
             Palestra                   NaN        1.0
             Podcast                    1.0        2.0
Igor         Alura+                     NaN        1.0
             Artigo                     NaN        1.0
             Curso                      NaN        1.0
             Podcast                    NaN        2.0
João         Artigo                     NaN        1.0
             Curso (E)                  NaN        2.0
             Formação                   1.0        NaN
             Podcast                    NaN        2.0
Marcelo      Artigo                     NaN        4.0
             Curso                      NaN        1.0
             Demanda Interna            1.0        NaN
             Podcast                    1.0        1.0
Mirla        Demanda Interna            1.0        NaN
             Formação                   1.0        NaN
             Podcast                    NaN        1.0
Val          Curso                      1.0        2.0
             Demanda Interna            1.0        NaN
             Palestra                   NaN        1.0
             Podcast                    NaN        2.0

In [ ]:
from datetime import datetime, timedelta

def parse_date(date_str):
    # Tenta converter a data usando diferentes formatos
    for fmt in ('%Y-%m-%d', '%Y-%m-%dT%H:%M:%SZ'):
        try:
            return datetime.strptime(str(date_str), fmt).strftime('%Y-%m-%d')
        except ValueError:
            continue
    # Retorna None se nenhum formato corresponder
    return ''

In [ ]:
start = '2023-03-15'
end = '2023-03-20'
current = '2023-03-15T13:00:00Z'

In [ ]:
parse_date(start)

'2023-03-15'

In [ ]:
parse_date(end)

'2023-03-20'

In [ ]:
parse_date(current)

'2023-03-15'

In [ ]:
parse_date(123)

''

In [ ]:
teste = "Evento Quinta [2024-04-01]"

In [ ]:
from datetime import timedelta, datetime, date

year, month, day = [int(item) for item in teste.split()[-1].replace('[', '').replace(']', '').split('-')]
str(date(year, month, day) - timedelta(days=6))

'2024-03-26'

In [ ]:
teste.split()[-1].replace('[', '').replace(']', '')

'2024-04-01'

In [ ]:
import ast

ast.literal_eval(teste.split()[-1])

SyntaxError: leading zeros in decimal integer literals are not permitted; use an 0o prefix for octal integers (<unknown>, line 1)